# Frontier League Stuff+ Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings

warnings.simplefilter(action='ignore')

df = pd.read_csv('fl_data_25.csv')
df.head()

,Unnamed: 0,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,...,Swing,Chase,FieldSide,PitchSource,PitchClass,xDamage,xwOBA,xSLG,xBA,barrel
0,1,1,2025-05-16,19:10:08.62,1.0,1.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,False,False,NaN,Fastball,Fastball,NaN,NaN,NaN,NaN,NaN
1,2,2,2025-05-16,19:10:20.59,1.0,2.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,True,False,Right,Fastball,Fastball,7.542667e-01,7.542667e-01,8.010000e-01,7.851667e-01,0.0
2,3,3,2025-05-16,19:10:55.20,2.0,1.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,True,True,Right,Fastball,Fastball,NaN,NaN,NaN,NaN,NaN
3,4,4,2025-05-16,19:11:17.11,2.0,2.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,True,False,Middle,Fastball,Fastball,NaN,NaN,NaN,NaN,NaN
4,5,5,2025-05-16,19:11:42.09,2.0,3.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,True,False,Middle,Fastball,Fastball,-1.887379e-16,-1.887379e-16,-3.963496e-16,-5.601075e-16,0.0


In [2]:
# Select features relevant for Stuff+
stuff_features = [
    'RelSpeed',      # Velocity
    'VertRelAngle', # Vertical release angle
    'HorzRelAngle', # Horizontal release angle
    'SpinRate',      # Spin rate
    'SpinAxis',      # Spin axis
    'VertBreak',     # Vertical movement
    'HorzBreak',     # Horizontal movement
    #'Tilt',          # Spin axis/tilt
    'Extension'      # Release extension (optional)
]

In [3]:
df_stuff = df[stuff_features]
df_stuff.head()

,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,VertBreak,HorzBreak,Extension
0,94.35239,-3.127214,-3.153577,2275.494571,220.925072,-16.41906,12.61398,5.61937
1,93.72900,-3.089860,-2.737261,2242.260452,213.981863,-14.00713,11.67560,5.66538
2,94.53260,-3.496708,-3.139473,2335.048720,220.236956,-14.05709,14.17646,5.62667
3,88.84878,-1.302765,-3.406383,2327.616719,220.699939,-28.47218,5.38654,5.53552
4,95.01096,-2.568358,-3.413276,2288.406975,218.918144,-13.22965,13.99891,5.50399


In [32]:
# Impute missing values
imputer = SimpleImputer(strategy='median')
stuff_data = imputer.fit_transform(df[stuff_features])

# Standardize features
scaler = StandardScaler()
stuff_scaled = scaler.fit_transform(stuff_data)

# Reduce to single Stuff+ value using PCA
pca = PCA(n_components=1)
stuff_plus_raw = pca.fit_transform(stuff_scaled).flatten()

# Normalize Stuff+ to league average = 100
stuff_plus = 100 + 15 * (stuff_plus_raw - np.mean(stuff_plus_raw)) / np.std(stuff_plus_raw)

# Add Stuff+ to the dataframe
df['StuffPlus'] = stuff_plus

# Save results
df.to_csv('fl_data_25_with_stuffplus.csv', index=False)

# Show sample pitches with Stuff+
print(df[['RelSpeed', 'SpinRate', 'VertBreak', 'HorzBreak', 'Tilt', 'Extension', 'StuffPlus']].head(10))

   RelSpeed     SpinRate  VertBreak  HorzBreak  Tilt  Extension   StuffPlus
0  94.35239  2275.494571  -16.41906   12.61398  1:15    5.61937  121.951323
1  93.72900  2242.260452  -14.00713   11.67560  1:15    5.66538  121.380933
2  94.53260  2335.048720  -14.05709   14.17646  1:15    5.62667  124.000328
3  88.84878  2327.616719  -28.47218    5.38654  1:15    5.53552  108.422600
4  95.01096  2288.406975  -13.22965   13.99891  1:15    5.50399  122.313259
5  94.20357  2320.867354  -13.39457   12.89367  1:15    5.73808  119.567639
6  88.18693  2251.904086  -26.48092    8.85969  1:30    5.85111  109.354182
7  94.56203  2319.665925  -13.33977   13.42780  1:15    5.66439  121.731801
8  94.42174  2279.442188  -16.15402   14.78461  1:30    5.66585  120.854580
9  94.76023  2348.932160  -15.66147   12.00024  1:15    5.68731  117.058423


In [33]:
df_stuffplus = pd.read_csv('fl_data_25_with_stuffplus.csv')
df_stuffplus.head()

,Unnamed: 0,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,...,Chase,FieldSide,PitchSource,PitchClass,xDamage,xwOBA,xSLG,xBA,barrel,StuffPlus
0,1,1,2025-05-16,19:10:08.62,1.0,1.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,False,NaN,Fastball,Fastball,NaN,NaN,NaN,NaN,NaN,121.951323
1,2,2,2025-05-16,19:10:20.59,1.0,2.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,False,Right,Fastball,Fastball,7.542667e-01,7.542667e-01,8.010000e-01,7.851667e-01,0.0,121.380933
2,3,3,2025-05-16,19:10:55.20,2.0,1.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,True,Right,Fastball,Fastball,NaN,NaN,NaN,NaN,NaN,124.000328
3,4,4,2025-05-16,19:11:17.11,2.0,2.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,False,Middle,Fastball,Fastball,NaN,NaN,NaN,NaN,NaN,108.422600
4,5,5,2025-05-16,19:11:42.09,2.0,3.0,"Sittinger, Brandyn",670087.0,Right,Lake Erie Crushers,...,False,Middle,Fastball,Fastball,-1.887379e-16,-1.887379e-16,-3.963496e-16,-5.601075e-16,0.0,122.313259


Top 10 Pitches by Stuff+

In [28]:
df_stuffplus[['Pitcher', 'PitcherTeam', 'PitchSource', 'RelSpeed', 'SpinRate', 'VertBreak', 'HorzBreak', 'Tilt', 'Extension', 'StuffPlus']].groupby(['Pitcher', 'PitcherTeam', 'PitchSource']).mean().sort_values(by='StuffPlus', ascending=False).head(10)

,,,RelSpeed,SpinRate,VertBreak,HorzBreak,Extension,StuffPlus
Pitcher,PitcherTeam,PitchSource,,,,,,
"Thiels, Brenton",Mississippi Mud Monsters,Sinker,92.187560,2071.284648,-17.084810,12.619610,6.692650,124.000000
"Stil, Matt",Sussex County Miners,Sinker,94.288327,2257.672000,-15.332855,16.381708,6.860706,123.705882
"Pfeifer, Rane",Evansville Otters,Sinker,95.328705,2333.666116,-17.785660,16.469415,5.943740,122.500000
"Stil, Matt",Sussex County Miners,Four-Seam,93.560109,2294.355255,-11.861415,12.006551,6.925015,122.033445
"Duby, Billy",Tri-City ValleyCats,Four-Seam,91.009475,2231.576647,-16.609400,12.319065,6.455705,122.000000
"Sanchez, Sergio",Lake Erie Crushers,Fastball,93.679871,2410.163065,-15.241306,14.268260,6.207704,121.875000
"Dieguez, Ryan",New Jersey Jackals,Fastball,92.025859,2136.053613,-16.451505,13.974768,6.155732,121.666667
"Pfeifer, Rane",Evansville Otters,Four-Seam,94.713173,2237.685759,-12.853183,10.438484,6.012634,121.615385
"Rodriguez, Luis",Trois-Rivieres Aigles,Four-Seam,93.777000,2294.760031,-11.012581,9.179495,6.092320,121.582043


Bottom 10 Pitches by Stuff+

In [29]:
df_stuffplus[['Pitcher', 'PitcherTeam', 'PitchSource', 'RelSpeed', 'SpinRate', 'VertBreak', 'HorzBreak', 'Tilt', 'Extension', 'StuffPlus']].groupby(['Pitcher', 'PitcherTeam', 'PitchSource']).mean().sort_values(by='StuffPlus', ascending=True).head(10)

RelSpeed     SpinRate  \
Pitcher          PitcherTeam           PitchSource                           
Quirion, Anthony Quebec Capitales      Curveball    50.610398  1601.643470   
Brannen, Cole    Gateway Grizzlies     Changeup     49.003395  1304.471582   
                                       Curveball    51.893325  1365.325314   
Novak, Kyle      Tri-City ValleyCats   Curveball    58.626570  2110.408229   
Harlan, Trotter  Down East Bird Dawgs  Changeup     54.051107  1238.634201   
Wargo, Tate      Gateway Grizzlies     Four-Seam    54.255510  1361.991724   
Fedko, Christian Schaumburg Boomers    Curveball    56.959405  1399.790767   
Novis, Tino      Ottawa Titans         Slider       61.707940   989.494912   
Marrero, Alan    Trois-Rivieres Aigles Curveball    57.977773  1478.142485   
Zeisler, Hank    Florence Y'alls       Slider       62.543330  3572.882113   

                                                     VertBreak  HorzBreak  \
Pitcher          PitcherTeam           PitchSource                          
Quirion, Anthony Quebec Capitales      Curveball   -122.242506  -1.612180   
Brannen, Cole    Gateway Grizzlies     Changeup    -117.849510   6.122540   
                                       Curveball   -111.970120   3.174420   
Novak, Kyle      Tri-City ValleyCats   Curveball    -91.974070  -4.098780   
Harlan, Trotter  Down East Bird Dawgs  Changeup     -97.320034   8.160598   
Wargo, Tate      Gateway Grizzlies     Four-Seam    -91.965250   1.333230   
Fedko, Christian Schaumburg Boomers    Curveball    -86.395240  -4.996830   
Novis, Tino      Ottawa Titans         Slider       -74.983070  -3.684140   
Marrero, Alan    Trois-Rivieres Aigles Curveball    -83.943737  -5.047593   
Zeisler, Hank    Florence Y'alls       Slider       -74.456540   0.548350   

                                                    Extension  StuffPlus  
Pitcher          PitcherTeam           PitchSource                        
Quirion, Anthony Quebec Capitales      Curveball     4.650188  28.888889  
Brannen, Cole    Gateway Grizzlies     Changeup      3.340675  30.500000  
                                       Curveball     3.453918  33.750000  
Novak, Kyle      Tri-City ValleyCats   Curveball     3.369980  43.000000  
Harlan, Trotter  Down East Bird Dawgs  Changeup      4.104949  47.130435  
Wargo, Tate      Gateway Grizzlies     Four-Seam     5.316680  48.000000  
Fedko, Christian Schaumburg Boomers    Curveball     6.634875  49.000000  
Novis, Tino      Ottawa Titans         Slider        5.663400  51.000000  
Marrero, Alan    Trois-Rivieres Aigles Curveball     4.881900  51.500000  
Zeisler, Hank    Florence Y'alls       Slider        4.485060  53.000000

Top 10 Bolts Pitches by Stuff+

In [30]:
Thunderbolts = df_stuffplus[df_stuffplus['PitcherTeam'] == 'Windy City ThunderBolts']
Thunderbolts[['Pitcher', 'PitcherTeam', 'PitchSource', 'RelSpeed', 'SpinRate', 'VertBreak', 'HorzBreak', 'Tilt', 'Extension', 'StuffPlus']].groupby(['Pitcher', 'PitcherTeam', 'PitchSource']).mean().sort_values(by='StuffPlus', ascending=False).head(10)

RelSpeed     SpinRate  \
Pitcher          PitcherTeam             PitchSource                           
Stants, Noah     Windy City ThunderBolts Sinker       91.344336  2121.109735   
                                         Four-Seam    91.867145  2152.856640   
Reynolds, Trevin Windy City ThunderBolts Sinker       92.573324  2062.679099   
                                         Four-Seam    93.111172  2240.560294   
Newman, Jacob    Windy City ThunderBolts Changeup     89.496512  1805.144701   
                                         Four-Seam    90.043793  2120.669849   
Pindel, Buddie   Windy City ThunderBolts Sinker       89.455659  2137.499043   
Evers, Aaron     Windy City ThunderBolts Four-Seam    92.432748  2247.117955   
Newman, Jacob    Windy City ThunderBolts Sinker       90.099953  2076.684801   
Pindel, Buddie   Windy City ThunderBolts Four-Seam    89.735359  2168.667962   

                                                      VertBreak  HorzBreak  \
Pitcher          PitcherTeam             PitchSource                         
Stants, Noah     Windy City ThunderBolts Sinker      -18.799460  15.416447   
                                         Four-Seam   -16.178146  12.139755   
Reynolds, Trevin Windy City ThunderBolts Sinker      -23.924825  15.561208   
                                         Four-Seam   -18.437415   9.936097   
Newman, Jacob    Windy City ThunderBolts Changeup    -22.768428  15.436542   
                                         Four-Seam   -17.986955  12.141172   
Pindel, Buddie   Windy City ThunderBolts Sinker      -20.307769  15.260775   
Evers, Aaron     Windy City ThunderBolts Four-Seam   -16.941825  13.672147   
Newman, Jacob    Windy City ThunderBolts Sinker      -20.167715  14.658613   
Pindel, Buddie   Windy City ThunderBolts Four-Seam   -17.695188  11.931513   

                                                      Extension   StuffPlus  
Pitcher          PitcherTeam             PitchSource                         
Stants, Noah     Windy City ThunderBolts Sinker        6.306855  120.172043  
                                         Four-Seam     6.300677  119.554545  
Reynolds, Trevin Windy City ThunderBolts Sinker        6.073285  117.688963  
                                         Four-Seam     6.105547  117.090909  
Newman, Jacob    Windy City ThunderBolts Changeup      5.608765  117.000000  
                                         Four-Seam     5.819254  116.884615  
Pindel, Buddie   Windy City ThunderBolts Sinker        5.829275  116.778711  
Evers, Aaron     Windy City ThunderBolts Four-Seam     5.579961  116.619718  
Newman, Jacob    Windy City ThunderBolts Sinker        5.708402  116.526882  
Pindel, Buddie   Windy City ThunderBolts Four-Seam     5.868499  116.471591

Bottom 10 Bolts Pitches by Stuff+

In [31]:
Thunderbolts[['Pitcher', 'PitcherTeam', 'PitchSource', 'RelSpeed', 'SpinRate', 'VertBreak', 'HorzBreak', 'Tilt', 'Extension', 'StuffPlus']].groupby(['Pitcher', 'PitcherTeam', 'PitchSource']).mean().sort_values(by='StuffPlus', ascending=True).head(10)

,,,RelSpeed,SpinRate,VertBreak,HorzBreak,Extension,StuffPlus
Pitcher,PitcherTeam,PitchSource,,,,,,
"Curpa, Jose",Windy City ThunderBolts,Curveball,60.339270,1478.236523,-77.509705,3.058490,4.601610,63.000000
"Rotz, Jeff",Windy City ThunderBolts,Curveball,75.767296,2096.240409,-62.767861,-13.438703,5.753617,67.060241
"Kirkeby, Dylan",Windy City ThunderBolts,Curveball,73.969675,2365.898443,-61.684690,-11.922974,5.046173,67.267442
"Vath, Bobby",Windy City ThunderBolts,Curveball,73.111047,2139.492293,-60.238545,-5.342913,5.018936,67.620690
"Savino, Dylan",Windy City ThunderBolts,Curveball,78.983206,2679.903089,-55.652549,-19.251137,4.614710,67.816327
"Kuzemka, Christian",Windy City ThunderBolts,Curveball,66.848330,1657.625982,-61.919360,3.047720,4.199120,71.000000
"Wehrle, Tyler",Windy City ThunderBolts,Curveball,76.609514,2669.808808,-53.106965,-15.558083,4.993115,71.454545
"Evans, Jalen",Windy City ThunderBolts,Curveball,77.498890,1861.251370,-52.069440,-4.313740,4.991205,71.500000
"Duncan, Greg",Windy City ThunderBolts,Curveball,79.408243,2937.393770,-55.664552,-11.960604,4.698755,71.849398
